## GPUS
- need a gpu to run (multiple)
- default storage and device is cpu for pytorch
- computations need to occur on same machine

In [1]:
!pip install d2l==1.0.3

In [2]:
import torch
from torch import nn
from d2l import torch as d2l

In [5]:
def cpu():  #@save
    """Get the CPU device."""
    return torch.device('cpu')

def gpu(i=0):  #@save
    """Get a GPU device."""
    return torch.device(f'cuda:{i}')

cpu(), gpu(), gpu(1)

(device(type='cpu'),
 device(type='cuda', index=0),
 device(type='cuda', index=1))

In [7]:
def num_gpus(): #@save
  return torch.cuda.device_count()

num_gpus()

1

In [11]:
def try_gpu(i=0):  #@save
    """Return gpu(i) if exists, otherwise return cpu()."""
    if num_gpus() >= i + 1:
        return gpu(i)
    return cpu()

def try_all_gpus():  #@save
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    return [gpu(i) for i in range(num_gpus())]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [23]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [24]:
Y = torch.rand(2, 3, device=try_gpu(0))
Y

tensor([[0.1835, 0.3586, 0.6268],
        [0.2181, 0.0695, 0.5088]], device='cuda:0')

In [25]:
# need to move X to gpu as well!!
# Transferring data is slow!!
# as a rule, many small operations worse than 1 big one.
# many operations at a time much better than many single operations interspred through code
# printing moves to main memory and converts to numpy!!
# X + Y  fails
Z = X.cuda(0)
print(Z.cuda(0) is Z)
Z + Y

True


tensor([[1.1835, 1.3586, 1.6268],
        [1.2181, 1.0695, 1.5088]], device='cuda:0')

In [30]:
net = nn.Sequential(nn.LazyLinear(1))
net = net.to(device=try_gpu())
net(X)


tensor([[0.1618],
        [0.1618]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [31]:
net[0].weight.data.device

device(type='cuda', index=0)

In [32]:
# ensure data and params in same device
@d2l.add_to_class(d2l.Trainer)  #@save
def __init__(self, max_epochs, num_gpus=0, gradient_clip_val=0):
    self.save_hyperparameters()
    self.gpus = [d2l.gpu(i) for i in range(min(num_gpus, d2l.num_gpus()))]

@d2l.add_to_class(d2l.Trainer)  #@save
def prepare_batch(self, batch):
    if self.gpus:
        batch = [a.to(self.gpus[0]) for a in batch]
    return batch

@d2l.add_to_class(d2l.Trainer)  #@save
def prepare_model(self, model):
    model.trainer = self
    model.board.xlim = [0, self.max_epochs]
    if self.gpus:
        model.to(self.gpus[0])
    self.model = model